In [20]:
from foreduce.tptp.parser import normalform
from foreduce.tptp.lexer import tptplexer

example = tptplexer.parse("""
    fof(4, axiom, (p & q) | r).
""")

In [21]:
example = normalform(example)
print(example.pretty())

tptp_file
  formula
    cnf
      4_0
      axiom
      cnf_formula
        disjunction
          fof_atom	p
          fof_atom	r
  formula
    cnf
      4_1
      axiom
      cnf_formula
        disjunction
          fof_atom	q
          fof_atom	r



In [19]:
import os
from tqdm import tqdm
from lark.exceptions import UnexpectedInput, UnexpectedToken, UnexpectedCharacters, UnexpectedEOF

results = []
success, failures = [], []
tptp_path = "/home/apluska/TPTP-v8.2.0/"
for file in tqdm(os.listdir(tptp_path + "Problems/PUZ/")):
    with open(tptp_path + "Problems/PUZ/" + file, "r") as f:
        data = f.read()
    try:
        result = tptplexer.parse(data)
    except (UnexpectedInput, UnexpectedToken, UnexpectedCharacters, UnexpectedEOF) as e:
        failures.append(file)
        continue
    success.append(file)
    results.append(result)
len(results)

  0%|          | 0/243 [00:00<?, ?it/s]

100%|██████████| 243/243 [00:34<00:00,  6.95it/s]


153

In [23]:
print(results[3].pretty())

tptp_file
  formula
    fof
      background
      axiom
      fof_formula
        fof_unary
          fof_quantified_formula
            ?
            A
            B
            C
            D
            E
            fof_unary
              fof_formula
                fof_binary
                  fof_unary
                    fof_atom
                      parent
                      fof_term	A
                  &
                  fof_binary
                    fof_unary
                      fof_atom
                        relation
                        fof_term	A
                        fof_term	of
                        fof_term	'Oedipus'
                    &
                    fof_binary
                      fof_unary
                        fof_atom
                          fof_term	'Iokaste'
                          =
                          fof_term	A
                      &
                      fof_binary
                        fof_unary
                    

In [24]:
print(normalform(results[3]).pretty())

tptp_file
  formula
    cnf
      background_0
      axiom
      cnf_formula
        disjunction
          fof_atom
            parent
            fof_term	sk0
  formula
    cnf
      background_1
      axiom
      cnf_formula
        disjunction
          fof_atom
            relation
            fof_term	sk0
            fof_term	of
            fof_term	'Oedipus'
  formula
    cnf
      background_2
      axiom
      cnf_formula
        disjunction
          fof_atom
            fof_term	'Iokaste'
            =
            fof_term	sk0
  formula
    cnf
      background_3
      axiom
      cnf_formula
        disjunction
          fof_atom
            parent
            fof_term	sk1
  formula
    cnf
      background_4
      axiom
      cnf_formula
        disjunction
          fof_atom
            relation
            fof_term	sk1
            fof_term	of
            fof_term	'Polyneikes'
  formula
    cnf
      background_5
      axiom
      cnf_formula
        disjunction
          

In [26]:
from lark import Token

Token("FOF", "fof").value

'fof'

In [ ]:
from itertools import chain
from lark import Transformer_InPlace, Tree

class Include(Transformer_InPlace):
    def __init__(self, include_path='/'):
        self.include_path = include_path
    
    def include(self, tree):
        with open(self.include_path + tree.children[0].value, "r") as f:
            data = f.read()
        input = tptplexer.parse(data)
        Include(include_path=self.include_path).transform(input)
        return Tree("include", [i for i in input.children if input.data == "formula"])
    
    def tptp_file(self, children):
        formulas  = [formula for formula in children if formula.data == "formula"]
        included = list(chain(*[include.children for include in children if include.data == "include"]))
        return Tree("tptp_file", formulas + included)
        

